In [1]:
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from sklearn import ensemble
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
fileName='./BODY_dataset.pickle'
with open(fileName,'rb') as f:
    data=pickle.load(f)
    dataset=data['features']
    label_shoulder=data['label_1']
    label_sleeve=data['label_2']
    del data
    f.close()

In [3]:
#used duplicate dataset to be suffled
dataset_1=dataset
dataset_2=dataset

In [4]:
#Using Regression for Shoulder breadth prediction

#suffle the dataset and label_shoulder
def randomize_data(dataset,labels):
    permutation=np.random.permutation(labels.shape[0])
    dataset=dataset[permutation, :]
    labels=labels[permutation]
    return dataset.astype(np.float32),labels.astype(np.float32)


In [7]:
dataset_1,label_shoulder=randomize_data(dataset_1,label_shoulder)

print(dataset_1.shape)
print(label_shoulder.shape)

batch_offset=int(dataset_1.shape[0] * 0.9)
train_dataset=dataset_1[:batch_offset, :]
train_labels=label_shoulder[:batch_offset]

test_dataset=dataset_1[batch_offset: , :]
test_labels=label_shoulder[batch_offset: ]

print("train_dataset :",train_dataset.shape)
print("test_dataset :",test_dataset.shape)

(1078, 2)
(1078,)
train_dataset : (970, 2)
test_dataset : (108, 2)


In [16]:
reg = ensemble.GradientBoostingRegressor(n_estimators=500, max_depth=4,min_samples_leaf=1, min_samples_split=2,learning_rate=0.01, loss='ls')
reg.fit(train_dataset,train_labels)
pred = reg.predict(test_dataset)
mse = mean_squared_error(test_labels, pred)
print("MSE: %.4f" % mse)

MSE: 268.8535


In [22]:
example=np.array([['1676','812']])
shoulder_breadth=reg.predict(example)
print ("Predicted shoulder breadth (in mm) :", shoulder_breadth)

Predicted shoulder breadth (in mm) : [ 426.4618453]


In [23]:
#using Regression again for Sleeve Length

#suffle the dataset and label_shoulder
def randomize_data(dataset,labels):
    permutation=np.random.permutation(labels.shape[0])
    dataset=dataset[permutation, :]
    labels=labels[permutation]
    return dataset.astype(np.float32),labels.astype(np.float32)

dataset_2,label_sleeve=randomize_data(dataset_2,label_sleeve)

print(dataset_2.shape)
print(label_sleeve.shape)



(1078, 2)
(1078,)


In [25]:
batch_offset=int(dataset_2.shape[0] * 0.9)
train_dataset=dataset_2[:batch_offset, :]
train_labels=label_sleeve[:batch_offset]

test_dataset=dataset_2[batch_offset: , :]
test_labels=label_sleeve[batch_offset: ]

print("train_dataset :",train_dataset.shape)
print("test_dataset :",test_dataset.shape)

train_dataset : (970, 2)
test_dataset : (108, 2)


In [26]:
reg = ensemble.GradientBoostingRegressor(n_estimators=500, max_depth=4,min_samples_leaf=1, min_samples_split=2,learning_rate=0.01, loss='ls')
reg.fit(train_dataset,train_labels)
pred = reg.predict(test_dataset)
mse = mean_squared_error(test_labels, pred)
print("MSE: %.4f" % mse)

MSE: 131.2895


In [27]:
example=np.array([['1676','812']])
sleeve_length=reg.predict(example)
print ("Predicted sleeve length (in mm) :", sleeve_length)

Predicted sleeve length (in mm) : [ 444.84959851]


In [32]:
#read pickle file to a 4 feature matrix and size label
import pandas as pd
file='./MLD.csv'
data=pd.read_csv(file)
data.head(1)


,WEIGHT(N),HEIGHT(mm),HEAD CIRCUMFERENCE(mm),SHOULDER BREADTH(mm),HAND LENGTH(mm),CHEST CIRCUMFERENCE(mm),WAIST CIRCUMFERENCE(mm),SLEEVE LENGTH(mm),AGE (months)
0,256,1278,505,298,138,602,553,352.0,120


In [33]:
label_final=data['CHEST CIRCUMFERENCE(mm)']
feature=data.drop(['WEIGHT(N)','HEAD CIRCUMFERENCE(mm)','HAND LENGTH(mm)','CHEST CIRCUMFERENCE(mm)','AGE (months)'],axis=1)
del data
label_final=np.array(label_final)
feature=np.array(feature)
from six.moves import cPickle as pickle
import os
root='.'
pickle_file=os.path.join(root,'size_dataset.pickle')
with open(pickle_file,'wb') as f:
    save={
    'label_final':label_final,
    'features':feature,
    }
    pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
    f.close()


In [35]:
fileName='./size_dataset.pickle'
with open(fileName,'rb') as f:
    data=pickle.load(f)
    dataset=data['features']
    label_size=data['label_final']
    
    del data
    f.close()

In [36]:
#suffle the dataset and label_shoulder
def randomize_data(dataset,labels):
    permutation=np.random.permutation(labels.shape[0])
    dataset=dataset[permutation, :]
    labels=labels[permutation]
    return dataset.astype(np.float32),labels.astype(np.float32)

dataset,label_size=randomize_data(dataset,label_size)

print(dataset.shape)
print(label_size.shape)

batch_offset=int(dataset.shape[0] * 0.9)
train_dataset=dataset[:batch_offset, :]
train_labels=label_size[:batch_offset]

test_dataset=dataset[batch_offset: , :]
test_labels=label_size[batch_offset: ]

print("train_dataset :",train_dataset.shape)
print("test_dataset :",test_dataset.shape)

(1078, 4)
(1078,)
train_dataset : (970, 4)
test_dataset : (108, 4)


In [58]:
reg = ensemble.GradientBoostingRegressor(n_estimators=500, max_depth=3,min_samples_leaf=2, min_samples_split=3,learning_rate=0.01, loss='ls')
reg.fit(train_dataset,train_labels)
pred = reg.predict(test_dataset)
mse = mean_squared_error(test_labels, pred)
print("MSE: %.4f" % mse)

MSE: 850.6831


In [61]:
final_example=[['1676',shoulder_breadth,'812',sleeve_length]]
predicted_size=reg.predict(final_example)
print("predicted shirt size(in mm) :",predicted_size)

predicted shirt size(in mm) : [ 900.88344074]
